In [24]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Burundi_Code_2017_penal.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
# data

In [25]:
len(data)

128

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)
docs = text_splitter.split_documents(data)
# print(docs[200])
# print("================================================")
print("Total number of documents: ",len(docs))

Total number of documents:  824


In [27]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [28]:

vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [29]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("Selon l'article 1, qu'est-ce qu'une infraction ?")

In [30]:
len(retrieved_docs)

10

In [31]:
print(retrieved_docs[5].page_content)

Code.	
Art.19.‐	L’auteur	de	l’infraction	est	celui	qui	en	commet	personnelleme nt	les	différents	
éléments	matériels	et	intellectuels	tels	que	définis	par	la	loi.	
Art.20.‐	L’auteur	intellectuel	est	celui	qui	conçoit	l’infraction	et	fa it	réaliser	tous	ou	
certains	actes	matériels	par	un	tiers.	
Art.21.‐	 A	 l’exception	 de	 celles	 citées	 à	 l’article	 24,	 les	 personnes	 morales	 sont	
pénalement	 responsables	 des	 infractions	 commises	 par	 leurs	 dirigeants	 ou	
représentants	légaux	agissant	pour	le	compte	de	ces	personnes	o u	dans	la	défense	de


In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite",temperature=1, max_tokens=500)

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Vous êtes un assistant spécialisé dans la réponse aux questions concernant le Code pénal du Burundi."
    "Utilisez uniquement les extraits de contexte fournis pour formuler votre réponse "
    "Si la réponse n’est pas présente dans le contexte, indiquez que vous ne savez pas "
    "Répondez de manière claire, précise et en trois phrases maximum "
    "Réponse flexible dans ta manière de parler."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [49]:
response = rag_chain.invoke({"input": "Est-ce que la loi fait une différence entre voler un portefeuille dans un sac et cambrioler une maison en cassant une fenêtre ? Comment les punitions changent-elles ?"})
print(response["answer"])

Oui, le Code pénal du Burundi fait une distinction entre voler un portefeuille et cambrioler une maison. Voler un portefeuille peut être considéré comme un vol simple, tandis que cambrioler une maison avec effraction est un vol aggravé.

Les punitions varient en fonction des circonstances. Le vol simple est puni de peines spécifiques, tandis que le cambriolage, en raison de ses circonstances aggravantes, entraîne des peines plus sévères, allant jusqu'à la servitude pénale à perpétuité dans certains cas.

Le vol est aussi aggravé s'il est commis la nuit, dans un local d'habitation ou ses dépendances, ou avec l'usage d'effraction, d'escalade ou de fausses clés.
